In [1]:
import numpy as np
from numpy import linalg as la
from qutip import *

import mod_score as sc
import total_sys_GGMMs as ggmm

allGGMMs = ggmm.constructTotalGGMMs(2)

## $n = 2$ qubit case

In [27]:
def randHerm(n):
    rand_ham = rand_herm(2**n, density=1, dims=[[2]*n, [2]*n])
    rand_ham -= rand_ham.tr()/(2**n) 
    return rand_ham

def random_three_vector():
    """http://stackoverflow.com/questions/5408276/python-uniform-spherical-distribution"""
    phi = np.random.uniform(0,np.pi*2)
    costheta = np.random.uniform(-1,1)
    theta = np.arccos( costheta )
    x = np.sin( theta) * np.cos( phi )
    y = np.sin( theta) * np.sin( phi )
    z = np.cos( theta )
    return (x,y,z)

In [61]:
# Defining R matrices
def R(_α, _β, _Γ):
    _σ = [sigmax(), sigmay(), sigmaz(), qeye(2)]
    _ρ = tensor(_σ[_α], _σ[_β])
    return (_Γ*_ρ*_Γ.dag()).ptrace(0)

In [66]:
# Defining cost function
def cost(_params, _H):
    #_params should be of the form [n1, n2, n3, θ1, θ2, ... θ15]
    _τ = la.norm(_H, 2)
    _U = sc.construct_unitary(2, _params[3:-1], allGGMMs)
    _Γ = (-1j*_τ*_U*_H*_U.dag()).expm()
    
    quadTr = []
    for i in range(3):
        for j in range(3):
            t1 = (1/16)*R(i, 3, _Γ)*R(j, 3, _Γ)
            t2 = []
            for k in range(3):
                t2.append(R(i, k, _Γ)*R(j, k, _Γ))
            t2 = (1/48)*sum(t2)
            quadTr.append(_params[i]*_params[j]*(t1 + t2).tr())
    quadT = (1/16)*sum(quadTr)
    
    linTr = []
    for i in range(3):
        for j in range(3):
            linTr.append(_params[i]*(R(3, j, _Γ)*R(i, j, _Γ)).tr())
    linT = (1/24)*sum(linTr)
    
    consTr = []
    for i in range(3):
        consTr.append((R(3, i, _Γ)*R(3, i, _Γ)).tr())
    consT = (1/2) + (1/48)*sum(consTr)
    
    return 1 - (consT + linT + quadT)

In [67]:
tθ = np.random.uniform(0, 2*np.pi, 15)
tn = random_three_vector()
params = [*tn, *tθ]
rH = randHerm(2)
cost(params, rH)

(0.2997942399286144+1.1112743155733263e-18j)

In [68]:
l = 0
while l != 100:
    tθ = np.random.uniform(0, 2*np.pi, 15)
    tn = random_three_vector()
    params = [*tn, *tθ]
    rH = randHerm(2)
    score = cost(params, rH)
    print(f'{l}: {score}')
    l += 1

0: (0.4019466710584432-7.494404923268099e-18j)
1: (0.4125120415337912+1.80184441719537e-18j)
2: (0.4266711992569645-8.288706623227918e-19j)
3: (0.3814714792124694+7.488245777796846e-18j)
4: (0.37634880713783314-4.5545526245178864e-18j)
5: (0.32675077932879526+2.058378545787224e-18j)
6: (0.43483502759857284+6.278294540056826e-19j)
7: (0.37198756813159384-2.6711837032893136e-18j)
8: (0.17885065060563876-1.834541246982969e-18j)
9: (0.39119579463551113+2.2675125475711904e-18j)
10: (0.3539839353518145-2.805849708763916e-18j)
11: (0.3456929425483244-5.002473408964708e-18j)
12: (0.3650526454815215-1.5225627995688259e-18j)
13: (0.42825384817137435+1.811886274798214e-18j)
14: (0.4678362780866705-1.3320880585608279e-18j)
15: (0.35166834481163267+3.841589689187225e-18j)
16: (0.3901114110883398+1.2766200134613739e-18j)
17: (0.3519315637568483-4.667855183310476e-18j)
18: (0.3522820474623245+6.3450547929444625e-18j)
19: (0.43089310714222173-6.768310829821944e-20j)
20: (0.46571118274070067+4.89389175